<a href="https://colab.research.google.com/github/hklee207/Aiden_Lee/blob/main/insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
mirichoi0218_insurance_path = kagglehub.dataset_download('mirichoi0218/insurance')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
insurance = pd.read_csv('/kaggle/input/insurance/insurance.csv')
insurance

In [ ]:
insurance.head()

In [ ]:
insurance.info()

In [ ]:
insurance["region"].value_counts()

In [ ]:
insurance.describe()

In [ ]:
import matplotlib.pyplot as plt

insurance.hist(bins=50 , figsize=(12,8))
plt.show()

In [ ]:
import numpy as np

def shuffle_and_split_data(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = shuffle_and_split_data(insurance, 0.2)
len(train_set)

In [ ]:
len(test_set)

In [ ]:
from zlib import crc32

def is_id_in_test_set(identifier, test_ratio):
    return crc32(np.int64(identifier)) < test_ratio * 2**32

def split_data_with_id_hash(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: is_id_in_test_set(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
insurance_with_id = insurance.reset_index()  # adds an `index` column
train_set, test_set = split_data_with_id_hash(insurance_with_id, 0.2, "index")

In [ ]:
insurance["bmi_cat"] = pd.cut(insurance["bmi"],
                               bins=[0, 25, 30, 35, 45, np.inf],
                               labels=[1, 2, 3, 4, 5])


In [ ]:
insurance["bmi_cat"].value_counts().sort_index().plot.bar(rot=0, grid=True)
plt.xlabel("bmi")
plt.ylabel("number_of_people")
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

splitter = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
strat_splits = []

for train_index, test_index in splitter.split(insurance, insurance["bmi_cat"]):
    strat_train_set_n = insurance.iloc[train_index]
    strat_test_set_n = insurance.iloc[test_index]
    strat_splits.append([strat_train_set_n, strat_test_set_n])


In [ ]:
strat_train_set, strat_test_set = strat_splits[0]

In [ ]:
strat_train_set, strat_test_set = train_test_split(
    insurance, test_size=0.2, stratify=insurance["bmi_cat"], random_state=42)


In [ ]:
strat_test_set["bmi_cat"].value_counts() / len(strat_test_set)

In [ ]:
# extra code – computes the data for Figure 2–10

def bmi_cat_proportions(data):
    return data["bmi_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(insurance, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall %": bmi_cat_proportions(insurance),
    "Stratified %": bmi_cat_proportions(strat_test_set),
    "Random %": bmi_cat_proportions(test_set),
}).sort_index()
compare_props.index.name = "BMI Category"
compare_props["Strat. Error %"] = (compare_props["Stratified %"] /
                                   compare_props["Overall %"] - 1)
compare_props["Rand. Error %"] = (compare_props["Random %"] /
                                  compare_props["Overall %"] - 1)
(compare_props * 100).round(2)

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("bmi_cat", axis=1, inplace=True)

In [ ]:
insurance = strat_train_set.copy()

In [ ]:
insurance.plot(kind="scatter", x="bmi", y="charges", grid=True)
plt.show()

In [ ]:
insurance.plot(kind="scatter", x="age", y="charges", grid=True)
plt.show()